In [4]:
# installing libraries
!pip install numpy pandas matplotlib pandas-datareader
!pip install mplfinance
!pip install tensorflow scikit-learn
!pip install ccxt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 19.6 MB/s eta 0:00:00


In [1]:
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime, timedelta
import pytz
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential


In [5]:
# Define the cryptocurrency symbol
symbol = 'BTC-USD'

# Get the current date and time in UTC
end_time_utc = datetime.utcnow()

# Convert UTC time to Pakistani time
pakistan_tz = pytz.timezone('Asia/Karachi')
end_time_pakistani = pytz.utc.localize(end_time_utc).astimezone(pakistan_tz)

# Calculate the start time as 60 days ago from the current time
start_time_pakistani = end_time_pakistani - timedelta(days=60)

# Fetch historical data at 5-minute intervals for the last 60 days
data = yf.download(symbol, start=start_time_pakistani, end=end_time_pakistani, interval='5m')

# Convert the index (time) to Pakistani time
data.index = data.index.tz_convert(pakistan_tz)

# Ensure proper reshaping for LSTM model
close_values = data['Close'].values.reshape(-1, 1)

[*********************100%%**********************]  1 of 1 completed


In [7]:
print(data['Close'].values.shape)


(17160,)


In [6]:
# Training the LSTM model
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_values)

prediction_days = 60
x_train, y_train = [], []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x - prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

Epoch 1/25
535/535 [==============================] - 13s 11ms/step - loss: 0.0051
Epoch 2/25
535/535 [==============================] - 6s 11ms/step - loss: 0.0019
Epoch 3/25
535/535 [==============================] - 6s 11ms/step - loss: 0.0016
Epoch 4/25
535/535 [==============================] - 6s 12ms/step - loss: 0.0013
Epoch 5/25
535/535 [==============================] - 6s 11ms/step - loss: 0.0010
Epoch 6/25
535/535 [==============================] - 6s 12ms/step - loss: 8.7298e-04
Epoch 7/25
535/535 [==============================] - 6s 11ms/step - loss: 7.5742e-04
Epoch 8/25
535/535 [==============================] - 6s 12ms/step - loss: 6.8680e-04
Epoch 9/25
535/535 [==============================] - 6s 11ms/step - loss: 5.6526e-04
Epoch 10/25
535/535 [==============================] - 6s 11ms/step - loss: 5.3087e-04
Epoch 11/25
535/535 [==============================] - 6s 11ms/step - loss: 5.4687e-04
Epoch 12/25
535/535 [==============================] - 6s 11ms/step - l

In [14]:
# Predict the next 5 minutes
last_data = data['Close'].values[-prediction_days:].reshape(1, -1)
last_data = last_data.reshape(-1, 1)

last_data_scaled = scaler.transform(last_data)
last_data_scaled = last_data_scaled.reshape(1, prediction_days, 1)

next_5_minutes_prediction = model.predict(last_data_scaled)
next_5_minutes_price = scaler.inverse_transform(next_5_minutes_prediction)

# Update the data DataFrame with the predicted price for the next 5 minutes
next_5_minutes_time = data.index[-1] + timedelta(minutes=5)
data.loc[next_5_minutes_time] = next_5_minutes_price[0, 0]
print(data.loc[next_5_minutes_time])

1/1 [==============================] - 0s 22ms/step
Open         42031.542969
High         42031.542969
Low          42031.542969
Close        42031.542969
Adj Close    42031.542969
Volume       42031.542969
Name: 2023-12-16 17:25:00+05:00, dtype: float64


In [17]:
# Plot the updated data
fig = go.Figure()

# Add a trace for the actual closing prices (in blue)
fig.add_trace(go.Scatter(
    x=data.index,
    y=data['Close'],
    mode='markers+lines',
    name='Actual Prices',
    hovertemplate='%{x|%d %b, %Y :: %I:%M %p}<br>Price: $%{y:.2f}',
    line=dict(color='blue')
))

# Add a trace for the predicted prices (in red)
fig.add_trace(go.Scatter(
    x=[next_5_minutes_time],
    y=[next_5_minutes_price[0, 0]],
    mode='markers',
    name='Predicted Price',
    hovertemplate='%{x|%d %b, %Y :: %I:%M %p}<br>Price: $%{y:.2f}',
    marker=dict(color='red', size=10)
))

# Set x-axis properties
fig.update_xaxes(
    type='category',
    tickmode='linear',
    tick0=0,
    dtick=5 * 60 * 1000,  # 5 minutes in milliseconds
    tickformat='%H:%M'  # Format as HH:MM
)

# Set y-axis label
fig.update_yaxes(title_text='Price (USD)')

# Update layout for hover information
fig.update_layout(
    title='Bitcoin Price at 5-Minute Intervals with Predicted Price',
    hovermode='x unified'
)

# Display the figure
fig.show()
